In [4]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import talib as ta
import datetime, time
import pickle
from tqdm import tqdm_notebook

import pdblp as pb
import blpapi 

In [5]:
plt.rcParams['figure.figsize'] = [12, 8]
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'
plt.style.use('ggplot')

con = pb.BCon(port=8194, timeout=5000)
con.start()

In [6]:
df_code = pd.read_excel('data/etf_pair_code.xlsx', dtype={'symbol':str})
code_list = df_code.symbol.unique().tolist()
ticker_list = (df_code.symbol + ' ' + df_code.region).fillna('ita').unique().tolist()

In [7]:
df = pd.DataFrame()
err_code = []
tic = time.perf_counter()
fields = ['HIGH', 'OPEN', 'LOW', 'VOLUME', 'PX_LAST']
cols = ['High', 'Open', 'Low', 'Volume', 'Adj Close']
start = '20100101'
end = datetime.datetime.today().strftime('%Y%m%d')

本次需要测试的代码

In [ ]:
ticker_list.remove('HKD Curncy')
ticker_list.remove('ita')
temp_list = ticker_list[:20]
temp_df = con.bdh(temp_list, fields, start, end)

In [ ]:
def save_file(dataframe, name):
    with open(name, 'wb') as f:
        pickle.dump(dataframe, f)

save_file(temp_df, 'temp_dataframe')

请把上面测试得到temp_dataframe文件发给我

由于之前的test 2测试，已经把data数据破坏了；

所以还是需要重新跑一次，保存到csv中，需要说明的是，后续的更新代码不会采取下列方法

In [ ]:
for num,i in tqdm_notebook(enumerate(ticker_list)):
    if (num % 100 == 0) & (num != 0):
        time.sleep(5)
    try:
        temp_df = con.bdh(i, fields, start, end)
        temp_df = temp_df.loc[:, i]
        temp_df.columns = cols
        temp_df = temp_df.assign(code=i.split(' ')[0])

        try:
            df = pd.concat([df, temp_df], axis=0)
        except:
            print('concat data has problem, ticker code : %s' % i)

    except:
        print('retrieve %s data, have something wrong, index:%s' % (i, ticker_list.index(i)))
        err_code.append(i)
toc = time.perf_counter()

df.to_csv('data/data.csv')
print('save finish')
print('retrieve data cost time: %s' % (toc-tic))
